IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M1\NO_FAULT1


In [3]:
guasto = 0

VARIABILE RCOU POTENZA MOTORI

In [4]:
#andiamo a prendere una potenza media di 1400 per considerare il momento il cui inizia/finisce il volo
rcou_m1 = pd.read_csv("RCOU.csv")
rcou_m1 = rcou_m1.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1 = rcou_m1[((rcou_m1['C9'] >= potenza) & (rcou_m1['C10'] >= potenza) & (rcou_m1['C11'] >= potenza) & (rcou_m1['C12'] >= potenza) & (rcou_m1['C13'] >= potenza) & (rcou_m1['C14']>= potenza))]

rcou_m1=rcou_m1.reset_index(drop=True)

pd.set_option('display.max_rows',700)
display(rcou_m1)

,TimeUS,C9,C10,C11,C12,C13,C14
0,22664152,1360,1375,1382,1352,1371,1363
1,22676651,1382,1369,1360,1390,1364,1387
2,22771304,1452,1411,1456,1406,1473,1386
3,22782035,1507,1358,1409,1465,1479,1393
4,22792632,1519,1355,1384,1497,1465,1420
...,...,...,...,...,...,...,...
4180,68488846,1553,1460,1514,1503,1422,1584
4181,68499144,1536,1469,1506,1501,1397,1594
4182,68509358,1542,1448,1511,1482,1419,1565
4183,68523070,1572,1386,1463,1509,1415,1550


In [5]:
min=rcou_m1['TimeUS'][0]
max=rcou_m1['TimeUS'][len(rcou_m1)-1]
print(max)
print(min)

68533079
22664152


In [6]:
#xkf utilizzato per fare poi il confronto con i valori di roll, pitch e yaw in att
xkf1_m1 = pd.read_csv("XKF1_0.csv")
xkf1_m1 = xkf1_m1.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1 = xkf1_m1[((xkf1_m1['TimeUS'] >= min) & (xkf1_m1['TimeUS'] <= max))]
xkf1_m1 = xkf1_m1.reset_index(drop=True)
print(xkf1_m1)

        TimeUS  Roll  Pitch    Yaw
0     22675051  0.01  -0.44  13.24
1     22685153  0.03  -0.40  13.24
2     22695762 -0.03  -0.45  13.23
3     22705793 -0.04  -0.52  13.22
4     22715779  0.03  -0.40  13.23
...        ...   ...    ...    ...
4256  68486359 -4.46   1.19   5.63
4257  68496743 -4.40   1.35   5.58
4258  68507280 -4.34   1.52   5.53
4259  68521132 -4.14   1.73   5.49
4260  68531134 -3.95   1.91   5.45

[4261 rows x 4 columns]


In [7]:
att_m1 = pd.read_csv("ATT.csv")
att_m1 = att_m1.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1 = att_m1[((att_m1['TimeUS'] >= min) & (att_m1['TimeUS'] <= max))]
att_m1 = att_m1.reset_index(drop=True)
print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [8]:
for idx,i in enumerate(att_m1['Roll']):
    if(xkf1_m1['Roll'][idx] != i):
        att_m1['Roll'][idx] = ((att_m1['Roll'][idx] + xkf1_m1['Roll'][idx])/2)

for idx,i in enumerate(att_m1['Pitch']):
    if(xkf1_m1['Pitch'][idx] != i):
        att_m1['Pitch'][idx] = ((att_m1['Pitch'][idx] + xkf1_m1['Pitch'][idx])/2)

for idx,i in enumerate(att_m1['Yaw']):
    if(xkf1_m1['Yaw'][idx] != i):
        att_m1['Yaw'][idx] = ((att_m1['Yaw'][idx] + xkf1_m1['Yaw'][idx])/2)

print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1 = pd.read_csv("ESC_0.csv")
esc_0_m1 = esc_0_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1 = esc_0_m1[((esc_0_m1['TimeUS'] >= min) & (esc_0_m1['TimeUS'] <= max))]
esc_0_m1=esc_0_m1.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})
print(esc_0_m1)
esc_1_m1 = pd.read_csv("ESC_1.csv")
esc_1_m1 = esc_1_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1 = esc_1_m1[((esc_1_m1['TimeUS'] >= min) & (esc_1_m1['TimeUS'] <= max))]
esc_1_m1=esc_1_m1.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1 = pd.read_csv("ESC_2.csv")
esc_2_m1 = esc_2_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1 = esc_2_m1[((esc_2_m1['TimeUS'] >= min) & (esc_2_m1['TimeUS'] <= max))]
esc_2_m1=esc_2_m1.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1 = pd.read_csv("ESC_3.csv")
esc_3_m1 = esc_3_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1 = esc_3_m1[((esc_3_m1['TimeUS'] >= min) & (esc_3_m1['TimeUS'] <= max))]
esc_3_m1=esc_3_m1.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1 = pd.read_csv("ESC_4.csv")
esc_4_m1 = esc_4_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1 = esc_4_m1[((esc_4_m1['TimeUS'] >= min) & (esc_4_m1['TimeUS'] <= max))]
esc_4_m1=esc_4_m1.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1 = pd.read_csv("ESC_5.csv")
esc_5_m1 = esc_5_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1 = esc_5_m1[((esc_5_m1['TimeUS'] >= min) & (esc_5_m1['TimeUS'] <= max))]
esc_5_m1=esc_5_m1.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})



       TimeUS  RPM(ESC_0)  Curr(ESC_0)
68   22742780        3495         2.16
69   22845457        4097         2.65
70   22959904        4696         3.34
71   23059012        5180         4.33
72   23167332        5753         4.29
73   23272089        6060         3.92
74   23387634        6310         4.54
75   23499927        6266         4.03
76   23602066        6227         3.98
77   23709666        6214         4.15
78   23821462        6291         4.02
79   23922931        6084         4.91
80   24140073        6394         5.41
81   24241826        6617         4.74
82   24355612        6651         4.40
83   24458193        6428         4.01
84   24563045        6225         3.88
85   24677142        6148         4.24
86   24780143        6059         3.99
87   24894895        6102         3.90
88   25000816        5953         4.13
89   25111147        6179         3.85
90   25216093        6107         3.97
91   25322971        6011         3.83
92   25431084        5965

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1 = pd.read_csv("IMU_0.csv")
imu_0_m1 = imu_0_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1 = imu_0_m1[((imu_0_m1['TimeUS'] >= min) & (imu_0_m1['TimeUS'] <= max))]

imu_1_m1 = pd.read_csv("IMU_1.csv")
imu_1_m1 = imu_1_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1 = imu_1_m1[((imu_1_m1['TimeUS'] >= min) & (imu_1_m1['TimeUS'] <= max))]

imu_2_m1 = pd.read_csv("IMU_2.csv")
imu_2_m1 = imu_2_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1 = imu_2_m1[((imu_2_m1['TimeUS'] >= min) & (imu_2_m1['TimeUS'] <= max))]

imu_m1 = pd.concat((imu_0_m1, imu_1_m1, imu_2_m1))
imu_m1=imu_m1.groupby(imu_m1.TimeUS, as_index=False).mean()

display(imu_m1)


,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,22666080,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081
1,22668252,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686
2,22670635,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578
3,22675772,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946
4,22681012,-0.035033,-0.112585,-0.025552,-0.159212,0.062739,-9.546370
...,...,...,...,...,...,...,...
16503,68522130,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947
16504,68524886,0.331998,0.249299,-0.035411,-0.695731,2.937416,-12.088990
16505,68527023,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677
16506,68530131,0.314659,0.326277,-0.025974,-0.837869,3.015614,-12.519287


In [21]:
last_cell = imu_m1.iloc[-1, imu_m1.columns.get_loc('TimeUS')]
print(last_cell)

68532323


In [33]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
print(to_be)

[0, 2857, 5714, 8571, 11428, 14285, 17142, 19999, 22856, 25713, 28570, 31427, 34284, 37141, 39998, 42855, 45712, 48569, 51426, 54283, 57140, 59997, 62854, 65711, 68568, 71425, 74282, 77139, 79996, 82853, 85710, 88567, 91424, 94281, 97138, 99995, 102852, 105709, 108566, 111423, 114280, 117137, 119994, 122851, 125708, 128565, 131422, 134279, 137136, 139993, 142850, 145707, 148564, 151421, 154278, 157135, 159992, 162849, 165706, 168563, 171420, 174277, 177134, 179991, 182848, 185705, 188562, 191419, 194276, 197133, 199990, 202847, 205704, 208561, 211418, 214275, 217132, 219989, 222846, 225703, 228560, 231417, 234274, 237131, 239988, 242845, 245702, 248559, 251416, 254273, 257130, 259987, 262844, 265701, 268558, 271415, 274272, 277129, 279986, 282843, 285700, 288557, 291414, 294271, 297128, 299985, 302842, 305699, 308556, 311413, 314270, 317127, 319984, 322841, 325698, 328555, 331412, 334269, 337126, 339983, 342840, 345697, 348554, 351411, 354268, 357125, 359982, 362839, 365696, 368553, 37

In [38]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
23983  68519431
23984  68522288
23985  68525145
23986  68528002
23987  68530859

[23988 rows x 1 columns]


In [39]:
#sincronizzazione dei tempi tra att e imu_2
m1_nofault = pd.merge_ordered(imu_m1,att_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22666080 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1      22668252  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
2      22670635  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
3      22675772  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
4      22675940  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
...         ...       ...       ...       ...       ...       ...        ...   
20764  68524886  0.331998  0.249299 -0.035411 -0.695731  2.937416 -12.088990   
20765  68527023  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
20766  68530131  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
20767  68532323  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
20768  68532571  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  Pitch  

In [40]:
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_nofault = pd.merge_ordered(m1_nofault, esc_0_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_1_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_2_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_3_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_4_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_5_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
m1_nofault = pd.merge_ordered(m1_nofault, rcou_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")

print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22664152 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1      22666080 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2      22668252  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
3      22668569  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4      22670635  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
...         ...       ...       ...       ...       ...       ...        ...   
27405  68527023  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
27406  68530131  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
27407  68532323  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27408  68532571  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27409  68533079  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  RP

In [41]:
index_to_remove=[]
for ind in att_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
print("--------------------")
print(len(index_to_remove))

4261
4671
5082
5492
5900
6305
6717
10902
--------------------
10902


In [42]:
#aggiornamento del tempo ponendo a 0 il momento in cui il volo inizia per noi e settaggio della variabile guasto
m1_nofault = m1_nofault[~m1_nofault.TimeUS.isin(index_to_remove)]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"] - m1_nofault.iloc[0]["TimeUS"]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"].astype(int)
m1_nofault["Guasto"] = guasto
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
1             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2          2172  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4          4555  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
5          9692  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
8         14932 -0.035033 -0.112585 -0.025552 -0.159212  0.062739  -9.546370   
...         ...       ...       ...       ...       ...       ...        ...   
27401  45856050  0.269569  0.293996 -0.016782 -0.722770  2.779444 -11.869947   
27404  45858806  0.331998  0.249299 -0.035411 -0.695731  2.937416 -12.088990   
27405  45860943  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
27406  45864051  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
27407  45866243  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  Cu

In [43]:
m1_nofault_final = pd.merge_ordered(m1_nofault,df)
m1_nofault_final=m1_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1          2172  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
2          2857  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
3          4555  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
4          5714  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
...         ...       ...       ...       ...       ...       ...        ...   
40486  68519431  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
40487  68522288  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
40488  68525145  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
40489  68528002  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
40490  68530859  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  Cu

In [44]:
m1_nofault_final = m1_nofault_final[m1_nofault_final.TimeUS.isin(to_be)]

In [45]:
m1_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
2,2857,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
4,5714,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
5,8571,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
7,11428,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40486,68519431,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
40487,68522288,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
40488,68525145,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
40489,68528002,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0


In [46]:
from datetime import timedelta
m1_nofault_final=m1_nofault_final.reset_index(drop=True)
m1_nofault_final['TimeUS'] = pd.to_datetime(m1_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
1,00:00:00.002857,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
2,00:00:00.005714,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
3,00:00:00.008571,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
4,00:00:00.011428,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23983,00:01:08.519431,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
23984,00:01:08.522288,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
23985,00:01:08.525145,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
23986,00:01:08.528002,0.326869,0.320070,-0.022980,-0.659711,3.074340,-12.657953,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0


## Analisi nel tempo e in frequenza

In [47]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [50]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_nofault_final)/350)):
        V1=m1_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab
    #display(tab)

In [51]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
#result = pd.merge(, how="right")
display(df_merged)


d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_19204\3363470071.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_19204\3363470071.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.041361,0.020029,-0.886667,0.147249,77.0,60.0,7.180687,3.427287,0.016129,...,10285.174377,3.230513,0.538919,-1.084396,3.312642,107.0,106.0,40.978871,6.818936,0
1,1.0,-0.041617,0.019995,-0.888193,0.147207,66.0,51.0,7.299761,3.630996,0.016036,...,10558.146490,3.232784,0.539200,-1.079465,3.314900,103.0,102.0,41.082744,6.924145,0
2,2.0,-0.041685,0.019939,-0.883305,0.147040,69.0,53.0,7.575255,3.612239,0.015941,...,10566.266379,3.235042,0.539468,-1.074524,3.317143,107.0,106.0,41.162716,6.985050,0
3,3.0,-0.041795,0.019887,-0.879490,0.146894,74.0,57.0,7.830195,3.434696,0.015770,...,10365.577701,3.237288,0.539725,-1.069570,3.319373,105.0,104.0,41.218940,7.002527,0
4,4.0,-0.042012,0.019848,-0.879415,0.146823,75.0,57.0,8.041255,3.246523,0.015616,...,10146.587402,3.239521,0.539971,-1.064606,3.321588,109.0,108.0,41.251584,6.977730,0
5,5.0,-0.042180,0.019802,-0.877450,0.146715,78.0,58.0,8.263713,3.602297,0.015531,...,9935.481386,3.241742,0.540205,-1.059629,3.323789,107.0,105.0,41.260832,7.120805,0
6,6.0,-0.042183,0.019746,-0.871464,0.146527,73.0,57.0,8.538466,3.700325,0.015382,...,9957.930608,3.243950,0.540428,-1.054640,3.325977,103.0,101.0,41.246881,7.280046,0
7,7.0,-0.042264,0.019693,-0.866937,0.146371,80.0,62.0,8.778712,3.472232,0.015075,...,9962.841512,3.246145,0.540640,-1.049639,3.328151,103.0,101.0,41.209943,7.441415,0
8,8.0,-0.042441,0.019649,-0.865358,0.146274,71.0,53.0,8.978353,3.261261,0.014900,...,9950.243597,3.248329,0.540841,-1.044625,3.330311,103.0,101.0,41.150240,7.597595,0
9,9.0,-0.042485,0.019595,-0.860027,0.146103,82.0,63.0,9.218787,3.487006,0.014692,...,9920.200523,3.250500,0.541032,-1.039598,3.332458,109.0,107.0,41.068009,7.741114,0


In [52]:
path_file = path_file.replace(r"M1\NO_FAULT1", "")
os.chdir(path_file)
df_merged.to_csv('m1_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
